In [1]:
import os

os.chdir('..')

#
os.chdir('Datasets')
print("Current Working Directory " , os.getcwd())

Current Working Directory  E:\MITB\Text Analytics\Project\Datasets\Submission_folder\Datasets


In [2]:
# Import Dataset
import pandas as pd


data_en=pd.read_csv("Songs_clean_en.csv")

In [ ]:
# remove duplicates lines function
def removeDuplicatesLines(k):
    lines_seen = list()
    for line in k:
        if line not in lines_seen:
            lines_seen.append(line)
    clean_lyric = '\n'.join(lines_seen)
    
    return clean_lyric

In [ ]:
# remove duplicates lines
data_en['Clean_lyrics'] = data_en['Lyrics'].apply(lambda x: removeDuplicatesLines(x.splitlines()))

In [ ]:
import numpy as np
import nltk
import re
import gensim

# NLTK Stop words
stop_list = nltk.corpus.stopwords.words('english')
# additional stop words
stop_list.extend(['ooh', 'la', 'hey', 'si', 'yeh', 'lo', 'c', 'da', 'ah', 'dat', 'yo', 'jah', 'dee', 'e', 'ya', 'je', 'ye', 'g', 'mi', 'dem', 'nuh', 'ba', 'yeah', 'el', 'wo', 'pon', 'le', 'oh', 'f', 'ta', 'gal', 'weh', 'eh', 'thee', 'te', 'en', 'les', 'ai', 'ca', 'ha', 'na', 'tu', 'de', 'di', 'thy', 'que', 'u', 'nah', 'un', 'yuh', 'inna', 'du', 'em', 'n', 'upon', 'fi'])

In [ ]:
# transform the DataFrame to list
docs = np.array(data_en["Clean_lyrics"]).tolist()
# tokenize the sentencs
docs1 = [nltk.word_tokenize(str(v)) for v in docs]

In [ ]:
#convert all the words to lower case
docs2 = [[w.lower() for w in doc] for doc in docs1]
# Use regular expression to keep only alphabetic words.
docs3 = [[w for w in doc if re.search('^[a-z]+$', w)] for doc in docs2]
# Remove stop words
docs4 = [[w for w in doc if w not in stop_list] for doc in docs3]
songs_docs = docs4

In [ ]:
# Create Dictionary
songs_dictionary = gensim.corpora.Dictionary(songs_docs)
# Create Corpus
songs_vecs = [songs_dictionary.doc2bow(doc) for doc in songs_docs]

In [ ]:
# find the optimal number of topics for LDA
from gensim.models import CoherenceModel

model_list = []
coherence_values = []
model_topics = []

for num_topics in range(4, 20, 2):
    songs_lda_x = gensim.models.ldamodel.LdaModel(corpus=songs_vecs, id2word=songs_dictionary, num_topics=num_topics)
    coherencemodel = CoherenceModel(model=songs_lda_x, texts=songs_docs, dictionary=songs_dictionary, coherence='c_v')
    model_topics.append(num_topics)
    model_list.append(songs_lda_x)
    coherence_values.append(coherencemodel.get_coherence())
    print("#Topics: " + str(num_topics) + " Score: " + str(coherencemodel.get_coherence()))

In [ ]:
# Show graph
import matplotlib.pyplot as plt

limit=20; start=4; step=2;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
#Save a model to disk
songs_lda_8=model_list[2]
songs_lda_8.save("songs_lda_8")

In [ ]:
#Code to save theta

mixture = [dict(songs_lda_8[x]) for x in songs_vecs]
pd.DataFrame(mixture).to_csv("doc_topic_mixture.csv")

# Write code here to save top 20 words for each topic (beta)
top_words_per_topic = []
for t in range(songs_lda_8.num_topics):
    top_words_per_topic.extend([(t, ) + x for x in songs_lda_8.show_topic(t, topn = 10)])

pd.DataFrame(top_words_per_topic, columns=['Topic', 'Word', 'P']).to_csv("top_words.csv")